In [6]:
import os
import pandas as pd
import tiktoken
import fitz  # PyMuPDF
import docx
import csv
from tqdm import tqdm  # 👈 import tqdm

tokenizer = tiktoken.get_encoding("cl100k_base")
max_tokens_per_chunk = 500

def chunk_text(text, max_tokens=500):
    tokens = tokenizer.encode(text)
    return [tokenizer.decode(tokens[i:i + max_tokens]) for i in range(0, len(tokens), max_tokens)]

def extract_text_from_pdf(file_path):
    doc = fitz.open(file_path)
    return [page.get_text() for page in doc]

def extract_text_from_docx(file_path):
    doc = docx.Document(file_path)
    return ["\n".join(para.text for para in doc.paragraphs)]

def extract_text_from_csv(file_path):
    with open(file_path, encoding="utf-8") as f:
        return [f.read()]

def extract_text_from_excel(file_path):
    df = pd.read_excel(file_path, sheet_name=None)
    pages = []
    for sheet, data in df.items():
        pages.append(f"Sheet: {sheet}\n" + data.to_string(index=False))
    return pages

def extract_text(file_path):
    ext = os.path.splitext(file_path)[1].lower()
    if ext == ".pdf":
        return extract_text_from_pdf(file_path)
    elif ext == ".docx":
        return extract_text_from_docx(file_path)
    elif ext in [".csv"]:
        return extract_text_from_csv(file_path)
    elif ext in [".xls", ".xlsx", ".xlsm"]:
        return extract_text_from_excel(file_path)
    else:
        print(f"Unsupported file type: {ext}")
        return []

def process_folder(folder_path):
    all_chunks = []
    file_list = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

    for filename in tqdm(file_list, desc="📄 Processing files"):
        file_path = os.path.join(folder_path, filename)

        try:
            pages = extract_text(file_path)
            for page_num, text in enumerate(pages, 1):
                chunks = chunk_text(text)
                for chunk_num, chunk in enumerate(chunks, 1):
                    all_chunks.append({
                        "file_name": filename,
                        "page_number": page_num,
                        "chunk_number": chunk_num,
                        "text": chunk
                    })
        except Exception as e:
            print(f"❌ Error processing {filename}: {e}")

    return pd.DataFrame(all_chunks)

# 🔍 Usage
folder_path = "/Users/yasir/Desktop/Project/Dr.X Files"
df = process_folder(folder_path)
print(df.head())
df.to_csv("all_chunked_text.csv", index=False)


/opt/anaconda3/envs/ai/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'Wedding budget'!$A:$K.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/opt/anaconda3/envs/ai/lib/python3.12/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/opt/anaconda3/envs/ai/lib/python3.12/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/opt/anaconda3/envs/ai/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
📄 Processing files: 100%|██████████████████████| 10/10 [00:00<00:00, 14.26it/s]

                              file_name  page_number  chunk_number  \
0  Dataset summaries and citations.docx            1             1   
1  Dataset summaries and citations.docx            1             2   
2  Dataset summaries and citations.docx            1             3   
3  Dataset summaries and citations.docx            1             4   
4    Ocean_ecogeochemistry_A_review.pdf            1             1   

                                                text  
0  Table 1. Description of studies included in th...  
1  bock, Texas. Agronomy Journal, 112(1), 148–157...  
2  2010). Soil Organic Carbon Input from Urban Tu...  
3   (2018). Soil carbon and nitrogen accumulation...  
4  327\nOceanography and Marine Biology: An Annua...  


In [38]:
import os
import time
import pandas as pd
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
import tiktoken
import ollama
import re
from tqdm import tqdm  # ✅ Added tqdm for progress bars

# ========== Embedding Setup ==========
embed_model = SentenceTransformer("all-MiniLM-L6-v2")
tokenizer = tiktoken.get_encoding("cl100k_base")
performance_log = []

# ========== Performance Logger ==========
def log_performance(stage, text_or_token_list, start_time, end_time):
    if isinstance(text_or_token_list, list) and isinstance(text_or_token_list[0], int):
        total_tokens = len(text_or_token_list)
    else:
        total_tokens = len(tokenizer.encode(text_or_token_list))

    elapsed = end_time - start_time
    tokens_per_sec = total_tokens / elapsed if elapsed > 0 else 0

    log_entry = {
        "stage": stage,
        "tokens": total_tokens,
        "duration_sec": round(elapsed, 4),
        "tokens_per_sec": round(tokens_per_sec, 2),
        "timestamp": pd.Timestamp.now()
    }

    performance_log.append(log_entry)
    print(f"📊 [{stage}] {total_tokens} tokens in {elapsed:.2f}s → {tokens_per_sec:.2f} tokens/sec")

    return tokens_per_sec

def export_log(filename="performance_log.csv"):
    df = pd.DataFrame(performance_log)
    df.to_csv(filename, index=False)
    print(f"✅ Performance log saved to {filename}")

def summarize_performance():
    df = pd.DataFrame(performance_log)
    if df.empty:
        print("⚠️ No performance data to summarize.")
        return
    summary = df.groupby("stage")["tokens_per_sec"].agg(["count", "min", "max", "mean"]).reset_index()
    print("\n📈 Performance Summary:")
    print(summary.to_string(index=False))
    return summary

# ========== Preprocessing DataFrame ==========
def remove_tables_and_numeric_lines(text):
    lines = text.splitlines()
    filtered_lines = []
    tables_and_numbers = []

    for line in lines:
        stripped = line.strip().lower()

        # Skip short chemical/symbol lines like "CO", "⇌", "+", etc.
        if len(line.strip()) < 10 and re.match(r"^[A-Za-z0-9()+−⇌\\s]*$", line):
            tables_and_numbers.append(line)
            continue
        # Remove lines like '.348136          NaN                                                              NaN'
        if re.fullmatch(r"[.\d\s]+nan\s*nan", line.strip().lower()):
            tables_and_numbers.append(line)
            continue
        # Remove lines like '.348136          NaN    NaN' or 'NaN                  0'
        if re.fullmatch(r"[.\d\s]*nan\s*\d*", line.strip().lower()):
            tables_and_numbers.append(line)
            continue


        # Skip lines that are too short and contain mostly numbers
        if len(line.strip()) < 40 and sum(c.isdigit() for c in line) / max(len(line), 1) > 0.3:
            tables_and_numbers.append(line)
            continue
        stripped = line.strip().lower()

        numbers = re.findall(r"\d", line)
        if len(numbers) / max(len(line), 1) > 0.4:
            tables_and_numbers.append(line)
            continue

        if re.search(r"\t", line) or re.search(r"\|", line):
            tables_and_numbers.append(line)
            continue

        if len(re.findall(r"\s{2,}", line)) > 2:
            tables_and_numbers.append(line)
            continue

        if any(keyword in stripped for keyword in ["author", "editor", "publisher", "published by", "phone", "email", "contact", "address"]):
            continue

        if re.search(r"\b\w+@\w+\.\w+\b", line):
            continue
        if re.search(r"\+?\d[\d\s().-]{7,}\d", line):
            continue

        filtered_lines.append(line)

    return "\n".join(filtered_lines), "\n".join(tables_and_numbers)
    
# ========== DataFrame Cleaning ==========
def clean_dataframe_text(df):
    df = df.dropna(subset=['text'])  # 🧹 Drop rows where 'text' is NaN
    df = df[df['text'].str.strip().astype(bool)]  # 🧹 Drop rows where 'text' is empty or only whitespace
    df = df[df['text'].str.len() >= 10]  # 🧹 Drop rows where 'text' is too short
    df = df.dropna(subset=['text'])  # 🧹 Drop rows where 'text' is NaN
    df = df.dropna(subset=['text'])  # 🧹 Drop rows where 'text' is NaN
    df = df[df['text'].str.strip().astype(bool)]  # 🧹 Drop rows where 'text' is empty or only whitespace
    df = df[df['text'].str.strip().astype(bool)]  # 🧹 Drop rows where 'text' is empty or only whitespace
    cleaned_rows = []
    for _, row in tqdm(df.iterrows(), total=len(df), desc="🧹 Cleaning text in DataFrame"):
        cleaned_text, _ = remove_tables_and_numeric_lines(row['text'])
        cleaned_rows.append({
            "file_name": row['file_name'],
            "page_number": row['page_number'],
            "chunk_number": row['chunk_number'],
            "text": cleaned_text
        })
        cleaned_df = pd.DataFrame(cleaned_rows)
    cleaned_df = cleaned_df[cleaned_df['text'].str.strip().astype(bool)]  # 🧹 Drop rows with empty cleaned text
    return cleaned_df



# ========== Embedding Function ==========
def embed_texts_local(texts):
    start = time.time()
    embeddings = embed_model.encode(texts).tolist()
    end = time.time()
    log_performance("embedding", " ".join(texts), start, end)
    return embeddings

# ========== Store to ChromaDB ==========
def store_df_in_chromadb_local(df, persist_dir="./chroma_storage", collection_name="cleaned_docs"):
    chroma_client = chromadb.PersistentClient(path=persist_dir)
    collection = chroma_client.get_or_create_collection(name=collection_name)

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="🔁 Storing in ChromaDB"):
        doc_id = f"{row['file_name']}_{row['page_number']}_{row['chunk_number']}"
        try:
            embedding = embed_texts_local([row['text']])[0]
            collection.add(
                documents=[row['text']],
                ids=[doc_id],
                metadatas=[{
                    "file_name": row['file_name'],
                    "page": row['page_number'],
                    "chunk": row['chunk_number']
                }],
                embeddings=[embedding]
            )
        except Exception as e:
            print(f"❌ Error at row {idx}: {e}")
    print("✅ Chunks stored in ChromaDB using local embeddings.")

# ========== Ask Mistral via Ollama ==========
def ask_mistral(question, context):
    prompt = f""""Write a clear, formal, human-like summary in academic tone.
    
Context:
{context}

Question:
{question}

Answer:"""
    start = time.time()
    response = ollama.chat(
        model="mistral",
        messages=[{"role": "user", "content": prompt}]
    )
    end = time.time()
    log_performance("RAG", prompt, start, end)
    return response['message']['content']

# ========== RAG Pipeline ==========
def rag_query(question, collection_name="cleaned_docs", persist_dir="./chroma_storage", top_k=1):
    query_vector = embed_texts_local([question])[0]

    chroma_client = chromadb.PersistentClient(path=persist_dir)
    collection = chroma_client.get_or_create_collection(name=collection_name)
    results = collection.query(query_embeddings=[query_vector], n_results=top_k)

    retrieved_chunks = results["documents"][0]
    context = "\n\n".join(retrieved_chunks)

    return ask_mistral(question, context)

# ========== Summarize All Documents ==========
def summarize_all_documents(collection_name="cleaned_docs", persist_dir="./chroma_storage"):
    """
    Summarize all chunks grouped by document ID.
    """
    chroma_client = chromadb.PersistentClient(path=persist_dir)
    collection = chroma_client.get_or_create_collection(name=collection_name)

    results = collection.get(include=["documents", "metadatas"])
    summaries_by_file = {}

    for doc, meta in zip(results["documents"], results["metadatas"]):
        file_name = meta.get("file_name", "unknown")
        text = doc if isinstance(doc, str) else doc[0]
        if file_name not in summaries_by_file:
            summaries_by_file[file_name] = []
        summaries_by_file[file_name].append(text)

    final_summaries = {}
    for file, chunks in tqdm(list(summaries_by_file.items())[:1], desc="🧠 Summarizing files"):
        combined_text = "\n\n".join(chunks)
        summary = ask_mistral("Summarize all the chunks of this document.", combined_text)
        final_summaries[file] = summary

    return final_summaries


In [40]:
clean_df= clean_dataframe_text(df)
store_df_in_chromadb_local(clean_df)
response = rag_query("what is Emotional intelligence?")
print("🤖 Mistral says:\n", response)


🔁 Storing in ChromaDB:   0%|                   | 1/407 [00:00<02:20,  2.89it/s]

📊 [embedding] 154 tokens in 0.25s → 618.58 tokens/sec
📊 [embedding] 185 tokens in 0.04s → 4325.06 tokens/sec
📊 [embedding] 32 tokens in 0.04s → 811.68 tokens/sec


🔁 Storing in ChromaDB:   2%|▎                  | 8/407 [00:00<00:23, 17.16it/s]

📊 [embedding] 276 tokens in 0.04s → 6534.55 tokens/sec
📊 [embedding] 442 tokens in 0.01s → 31564.58 tokens/sec
📊 [embedding] 249 tokens in 0.04s → 5795.81 tokens/sec
📊 [embedding] 498 tokens in 0.01s → 37721.69 tokens/sec
📊 [embedding] 500 tokens in 0.01s → 36640.44 tokens/sec
📊 [embedding] 143 tokens in 0.04s → 3839.78 tokens/sec


🔁 Storing in ChromaDB:   3%|▍                 | 11/407 [00:00<00:19, 20.06it/s]

...

🔁 Storing in ChromaDB:  99%|████████████████▋| 401/407 [00:12<00:00, 30.65it/s]

📊 [embedding] 386 tokens in 0.02s → 24373.37 tokens/sec
📊 [embedding] 499 tokens in 0.02s → 30182.25 tokens/sec
📊 [embedding] 457 tokens in 0.01s → 34237.08 tokens/sec
📊 [embedding] 286 tokens in 0.02s → 18859.10 tokens/sec
📊 [embedding] 166 tokens in 0.06s → 2756.93 tokens/sec


🔁 Storing in ChromaDB: 100%|█████████████████| 407/407 [00:13<00:00, 31.30it/s]

📊 [embedding] 86 tokens in 0.06s → 1463.61 tokens/sec
📊 [embedding] 273 tokens in 0.06s → 4584.67 tokens/sec
📊 [embedding] 138 tokens in 0.06s → 2397.84 tokens/sec
📊 [embedding] 4 tokens in 0.01s → 508.99 tokens/sec
📊 [embedding] 6 tokens in 0.01s → 801.61 tokens/sec
✅ Chunks stored in ChromaDB using local embeddings.


📊 [embedding] 5 tokens in 0.06s → 79.78 tokens/sec
📊 [RAG] 446 tokens in 51.23s → 8.71 tokens/sec
🤖 Mistral says:
  Emotional Intelligence (EI) refers to the ability to identify, use, understand, and manage emotions in positive ways to relieve stress, communicate effectively, empathize with others, overcome challenges, and defuse conflict. It involves self-awareness, self-regulation, motivation, empathy, and social skills (Goleman, 2011).

EI can be distinguished from Intelligence Quotient (IQ) as the latter measures cognitive abilities such as problem-solving, reasoning, and logic. EI is more concerned with emotional and interpersonal skills that affect how we interact with others and manage our own emotions effectively (Verywell Mind, 2023).

In this course, Computer Assisted Research Skills (CARS), students will not directly study EI as the primary focus. However, it is essential to note that understanding the basics of EI can provide valuable insights while conducting research and 

In [41]:
from rouge_score import rouge_scorer

# ========== Evaluate Summaries with ROUGE ==========
def evaluate_summaries_with_rouge(generated_summaries: dict, reference_summaries: dict):
    """
    Compare generated summaries to reference summaries using ROUGE scores.
    """
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    evaluation = {}

    for doc_id, generated in tqdm(generated_summaries.items(), desc="📏 Evaluating ROUGE"):
        reference = reference_summaries.get(doc_id, "")
        if not reference:
            print(f"⚠️ No reference summary found for document: {doc_id}")
            continue

        scores = scorer.score(generated, reference)
        evaluation[doc_id] = {
            metric: {
                "precision": round(scores[metric].precision, 4),
                "recall": round(scores[metric].recall, 4),
                "f1": round(scores[metric].fmeasure, 4)
            } for metric in scores
        }

    return evaluation


In [ ]:
response = summarize_all_documents()
# response = rag_query("Tell me about Emotional Intelligence")

print("🤖 Mistral says:\n", response)

In [48]:
# ========== Preloaded Reference Summaries ==========
reference_summaries = {
    "Dataset summaries and citations.docx": """
    1. Numerous studies have investigated carbon accumulation and nitrogen cycling in various urban land uses including residential soils, golf courses, and home lawns.
    2. Some research emphasizes specific biophysical factors influencing soil carbon, such as:
       - Microbial processes (e.g., Shi et al., 2012),
       - Management intensity and duration (e.g., Wang et al., 2014),
       - Historical land use changes (e.g., Raccanello et al., 2011).
    3. A subset of studies analyzes regional influences on carbon sequestration, examining how soil characteristics and climatic conditions affect carbon dynamics in urban ecosystems (e.g., Selhorst & Lal, 2011; Smith et al., 2018).
    4. Comparative assessments have been conducted to evaluate differences in carbon sequestration across urban land typologies, including turfgrass systems and residential zones with varied development histories.
    5. Findings indicate that urban landscapes can serve as significant carbon sinks, occasionally surpassing the sequestration potential of rural or forested lands (Selhorst & Lal, 2011; Trammell et al., 2020).
    6. Despite their potential benefits, urban environments also contribute to greenhouse gas emissions, especially from impervious surfaces and vehicular traffic (Townsend-Small & Czimczik, 2010).
    7. Research suggests that residential lawns have a variable carbon sequestration potential, heavily influenced by climate, management practices, and underlying soil properties.
    8. In conclusion, while urban landscapes offer valuable opportunities for climate change mitigation through carbon storage, maximizing their potential requires integrated land management strategies that simultaneously limit greenhouse gas emissions.
    """
}

# ========== Example Evaluation Run ==========
generated_summaries = summarize_all_documents()
results = evaluate_summaries_with_rouge(generated_summaries, reference_summaries)
data = results
for filename, scores in data.items():
    print(f"\nROUGE Scores for: {filename}\n")
    print(f"{'Metric':<10} {'Precision':<10} {'Recall':<10} {'F1 Score':<10}")
    for metric, values in scores.items():
        print(f"{metric.upper():<10} {values['precision']:<10.4f} {values['recall']:<10.4f} {values['f1']:<10.4f}")

🧠 Summarizing files: 100%|███████████████████████| 1/1 [00:24<00:00, 24.70s/it]


📊 [RAG] 679 tokens in 24.68s → 27.51 tokens/sec


📏 Evaluating ROUGE: 100%|████████████████████████| 1/1 [00:00<00:00, 62.27it/s]


ROUGE Scores for: Dataset summaries and citations.docx

Metric     Precision  Recall     F1 Score  
ROUGE1     0.3833     0.5337     0.4462    
ROUGE2     0.1150     0.1605     0.1340    
ROUGEL     0.1674     0.2331     0.1949    


In [ ]:
def summarize_all_documents(collection_name="docs", persist_dir="./chroma_storage", max_chunks=5):
    """
    Summarize a limited number of stored chunks to reduce load and return a list of summaries.
    """
    chroma_client = chromadb.PersistentClient(path=persist_dir)
    collection = chroma_client.get_or_create_collection(name=collection_name)

    results = collection.get(include=["documents"])
    all_chunks = results.get("documents", [])

    print(f"🧠 Summarizing up to {max_chunks} chunks...")
    summaries = []
    for idx, chunk in enumerate(all_chunks[:max_chunks]):
        if isinstance(chunk, str) and chunk.strip():
            summary = ask_mistral("Summarize the following document chunk.", chunk)
            summaries.append(summary)

    return summaries

In [ ]:
from sentence_transformers import SentenceTransformer

# Load once
embed_model = SentenceTransformer("all-MiniLM-L6-v2")

def embed_texts_local(texts):
    """Embed text using a local SentenceTransformer model."""
    return embed_model.encode(texts).tolist()


In [ ]:
import chromadb
import pandas as pd

def store_df_in_chromadb_local(df, persist_dir="./chroma_storage", collection_name="drx_docs"):
    chroma_client = chromadb.PersistentClient(path=persist_dir)
    collection = chroma_client.get_or_create_collection(name=collection_name)

    for idx, row in df.iterrows():
        doc_id = f"{row['file_name']}_{row['page_number']}_{row['chunk_number']}"
        try:
            embedding = embed_texts_local([row['text']])[0]
            collection.add(
                documents=[row['text']],
                ids=[doc_id],
                metadatas=[{
                    "file_name": row['file_name'],
                    "page": row['page_number'],
                    "chunk": row['chunk_number']
                }],
                embeddings=[embedding]
            )
        except Exception as e:
            print(f"❌ Error at row {idx}: {e}")
    print("✅ Chunks stored in ChromaDB using local embeddings.")


In [ ]:
def query_chromadb_local(question, collection_name="drx_docs", persist_dir="./chroma_storage", top_k=1):
    chroma_client = chromadb.PersistentClient(path=persist_dir)
    collection = chroma_client.get_or_create_collection(name=collection_name)

    query_vector = embed_texts_local([question])[0]
    results = collection.query(query_embeddings=[query_vector], n_results=top_k)
    # print(results)

    for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
        print(f"📄 {meta['file_name']} (Page {meta['page']}, Chunk {meta['chunk']})")
        print(doc)
        print("------")


In [ ]:
import ollama

def ask_mistral(question, context):
    prompt = f"""Answer the question based on the context below.

Context:
{context}

Question:
{question}

Answer:"""

    response = ollama.chat(
        model="mistral",
        messages=[{"role": "user", "content": prompt}]
    )
    return response['message']['content']


In [ ]:
def rag_query(question, collection_name="drx_docs", persist_dir="./chroma_storage", top_k=5):
    # Embed the question
    query_vector = embed_texts_local([question])[0]

    # Query ChromaDB
    chroma_client = chromadb.PersistentClient(path=persist_dir)
    collection = chroma_client.get_or_create_collection(name=collection_name)
    results = collection.query(query_embeddings=[query_vector], n_results=top_k)

    # Concatenate top chunks
    retrieved_chunks = results["documents"][0]
    context = "\n\n".join(retrieved_chunks)

    # Generate answer with Ollama Mistral
    answer = ask_mistral(question, context)
    return answer


In [ ]:
response = rag_query("summarize all papers?")
print("🤖 Mistral says:\n", response)